### Read CSV with top 100 songs bilboards

In [ ]:
import pandas as pd

bilboard_df1 = pd.read_csv("../datas/top100.csv", low_memory=False)
bilboard_df2 = pd.read_csv("../datas/top100oct.csv", low_memory=False)
bilboard_df1


In [ ]:
bilboard_df2

##### Compare 2 DF and print the duplicate items

In [ ]:
def duplicate_bilboards():
    for song, songs in enumerate(bilboard_df1['title']):
        for sound, sounds in enumerate(bilboard_df2['title']):
            if songs == sounds:
                playlist = {
                    "Musica": bilboard_df1['title'][sound],
                    "Artista": bilboard_df2['artist'][sound]
                }
                print(playlist)
duplicate_bilboards()

##### SparkSession create and Read csv

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
path = "../datas/top100oct.csv"
path2 = "../datas/top100.csv"
bilboard_df1_spark = spark.read.csv(path)
bilboard_df1_spark

##### Get all data with Schema context

In [ ]:
bilboard_df1_spark = spark.read.format("csv").option("inferSchema", "True").option("header", "True").csv(path)
bilboard_df1_spark.collect()

In [ ]:
bilboard_df2_spark = spark.read.format("csv").option("inferSchema", "True").option("header", "True").csv(path2)
bilboard_df2_spark.collect()

##### Show schema

In [ ]:
bilboard_df1_spark.printSchema()

In [ ]:
bilboard_df2_spark.printSchema()

In [ ]:
type(bilboard_df1_spark)

In [ ]:
type(bilboard_df1)

##### Get first 5 lines with array

In [ ]:
bilboard_df1_spark.take(5)

##### Format to show better

In [ ]:
display(bilboard_df1_spark.show(5))

In [ ]:
bilboard_df1_spark.count()

In [ ]:
bilboard_df2_spark.count()

##### Turn False inferSchema

In [2]:
bilboard_df1_spark = spark.read.format("csv").option("inferSchema", "False").option("header", "True").csv(path)
bilboard_df1_spark.collect()

NameError: name 'spark' is not defined

##### Pyspark code

In [1]:
from pyspark.sql.functions import *

bilboard_df1_spark.orderBy(expr("peak desc")).show(15)


NameError: name 'bilboard_df1_spark' is not defined

In [ ]:
bilboard_df1_spark.orderBy(expr("artist asc")).show(15)

In [ ]:
display(bilboard_df1_spark.select("artist").show(10))

#### Using filter + order by

In [ ]:
display(bilboard_df1_spark.filter("this_week_chart > 4").orderBy(expr("title asc")).show(10))

##### Rename column in data frame

In [ ]:
bilboard_df1_spark = bilboard_df1_spark.withColumnRenamed("artist", "Artista").withColumnRenamed("title", "Musica").withColumnRenamed(
    "this_week_chart", "TocadasNaSemana").withColumnRenamed("last_week_chart", "TocadasSemanaPassada").withColumnRenamed("peak", "Pico").withColumnRenamed("weeks_on_chart", "GraficoSemanal")
bilboard_df1_spark.show()


##### ADD Column in dataFrame

In [ ]:
bilboard_df1_spark = bilboard_df1_spark.withColumn("product", bilboard_df1_spark['Pico']*100).show(10)

##### Drop column

In [ ]:
#bilboard_df1_spark = bilboard_df1_spark.drop("Valor").show(10)

In [ ]:
display(bilboard_df1_spark.show(50))